In [18]:
import pandas as pd
import itertools
import warnings
import numpy as np
from sklearn import preprocessing
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import numpy as np
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Flatten
from tensorflow.keras.optimizers import SGD,Adam

from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn import metrics

from sklearn.feature_selection import RFE

from tensorflow import keras
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Flatten
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners.randomsearch import RandomSearch
from tensorflow.python.keras.metrics import Metric
from tensorflow.keras.callbacks import EarlyStopping
import time
import pickle
#import gin.tf


## Firstly we deal with the dataset "1 years before"

In [3]:
df=pd.read_csv("1 years before.csv")
df['final'][df['final']=='Bankrupted']=0
df['final'][df['final']=='Non-Bankrupted']=1
df.head()

bankruptcies=df['final'][df['final']==0].count()
rows=df.shape[0]
columns=df.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 25 columns. We have 49 bankrupted companies and 96 companies that still operate.


## Train test split and normalization

In [4]:
np.random.seed(333)

X=df.iloc[:,0:-1].values
Y=df.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

## Building the model

In [5]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(hp.Int("input_units",min_value=24,max_value=120,step=12),input_shape=(24,),activation='relu'))
    
    for i in range(hp.Int("n_layers",1,4)):
    
        model.add(Dense(hp.Int(f"dense_{i}_units",min_value=24,max_value=120,step=12),activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(Adam(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])

    return model

# Do NOT run this

In [20]:
LOG_DIR=f"first_year\{int(time.time())}"

tuner= RandomSearch(
    build_model,
    objective=kt.Objective("val_accuracy",direction='max'),
    max_trials=4,
    executions_per_trial=4,
    directory=LOG_DIR
)

earlyStopping =EarlyStopping(monitor='loss', patience=50, verbose=0, mode='min')
#mcp_save = ModelCheckpoint('mp.hdf5', save_best_only=True, monitor='loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1,min_delta=0.01, mode='min')

tuner.search(
    x=X_train,
    y=y_train,
    validation_data=(X_test,y_test),
    epochs=200,
    callbacks=[earlyStopping]
)

Train on 116 samples, validate on 29 samples
Epoch 1/200
116/116 [==============================] - ETA: 0s - loss: 0.6986 - accuracy: 0.40 - 1s 5ms/sample - loss: 0.6505 - accuracy: 0.5776 - val_loss: 0.6372 - val_accuracy: 0.6552
Epoch 2/200
116/116 [==============================] - ETA: 0s - loss: 0.5145 - accuracy: 0.75 - 0s 275us/sample - loss: 0.6448 - accuracy: 0.6638 - val_loss: 0.6099 - val_accuracy: 0.6552
Epoch 3/200
116/116 [==============================] - ETA: 0s - loss: 0.6789 - accuracy: 0.53 - 0s 155us/sample - loss: 0.5968 - accuracy: 0.6983 - val_loss: 0.6057 - val_accuracy: 0.6207
Epoch 4/200
116/116 [==============================] - ETA: 0s - loss: 0.6078 - accuracy: 0.65 - 0s 163us/sample - loss: 0.5795 - accuracy: 0.7069 - val_loss: 0.5914 - val_accuracy: 0.6552
Epoch 5/200
116/116 [==============================] - ETA: 0s - loss: 0.5112 - accuracy: 0.81 - 0s 180us/sample - loss: 0.5675 - accuracy: 0.7155 - val_loss: 0.5769 - val_accuracy: 0.6207
Epoch 6/200


Epoch 44/200
116/116 [==============================] - ETA: 0s - loss: 0.1293 - accuracy: 0.96 - 0s 163us/sample - loss: 0.1837 - accuracy: 0.9138 - val_loss: 2.5764 - val_accuracy: 0.6552
Epoch 45/200
116/116 [==============================] - ETA: 0s - loss: 0.1232 - accuracy: 0.96 - 0s 146us/sample - loss: 0.1280 - accuracy: 0.9655 - val_loss: 2.5468 - val_accuracy: 0.6552
Epoch 46/200
116/116 [==============================] - ETA: 0s - loss: 0.1672 - accuracy: 0.93 - 0s 163us/sample - loss: 0.1345 - accuracy: 0.9483 - val_loss: 2.6093 - val_accuracy: 0.5862
Epoch 47/200
116/116 [==============================] - ETA: 0s - loss: 0.1827 - accuracy: 0.93 - 0s 129us/sample - loss: 0.1451 - accuracy: 0.9397 - val_loss: 2.6809 - val_accuracy: 0.6552
Epoch 48/200
116/116 [==============================] - ETA: 0s - loss: 0.0597 - accuracy: 1.00 - 0s 138us/sample - loss: 0.0950 - accuracy: 0.9828 - val_loss: 2.7632 - val_accuracy: 0.6207
Epoch 49/200
116/116 [============================

Epoch 130/200
116/116 [==============================] - ETA: 0s - loss: 0.0028 - accuracy: 1.00 - 0s 155us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 5.9995 - val_accuracy: 0.5862
Epoch 131/200
116/116 [==============================] - ETA: 0s - loss: 0.0025 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0033 - accuracy: 1.0000 - val_loss: 6.0077 - val_accuracy: 0.5862
Epoch 132/200
116/116 [==============================] - ETA: 0s - loss: 0.0019 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 6.0545 - val_accuracy: 0.6207
Epoch 133/200
116/116 [==============================] - ETA: 0s - loss: 0.0035 - accuracy: 1.00 - 0s 146us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 6.0785 - val_accuracy: 0.6207
Epoch 134/200
116/116 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 1.00 - 0s 120us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 6.0868 - val_accuracy: 0.5862
Epoch 135/200
116/116 [======================

Epoch 173/200
116/116 [==============================] - ETA: 0s - loss: 6.6785e-04 - accuracy: 1.00 - 0s 155us/sample - loss: 0.0014 - accuracy: 1.0000 - val_loss: 6.5956 - val_accuracy: 0.5862
Epoch 174/200
116/116 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0015 - accuracy: 1.0000 - val_loss: 6.6056 - val_accuracy: 0.5862
Epoch 175/200
116/116 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.00 - 0s 138us/sample - loss: 0.0014 - accuracy: 1.0000 - val_loss: 6.6436 - val_accuracy: 0.6207
Epoch 176/200
116/116 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 1.00 - 0s 146us/sample - loss: 0.0013 - accuracy: 1.0000 - val_loss: 6.6652 - val_accuracy: 0.6207
Epoch 177/200
116/116 [==============================] - ETA: 0s - loss: 0.0020 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0013 - accuracy: 1.0000 - val_loss: 6.6673 - val_accuracy: 0.5862
Epoch 178/200
116/116 [==================

116/116 [==============================] - ETA: 0s - loss: 0.0484 - accuracy: 1.00 - 0s 181us/sample - loss: 0.0598 - accuracy: 0.9914 - val_loss: 3.5739 - val_accuracy: 0.6552
Epoch 59/200
116/116 [==============================] - ETA: 0s - loss: 0.1068 - accuracy: 0.93 - 0s 155us/sample - loss: 0.0906 - accuracy: 0.9741 - val_loss: 3.4982 - val_accuracy: 0.5172
Epoch 60/200
116/116 [==============================] - ETA: 0s - loss: 0.1531 - accuracy: 0.93 - 0s 155us/sample - loss: 0.0972 - accuracy: 0.9569 - val_loss: 3.6780 - val_accuracy: 0.6552
Epoch 61/200
116/116 [==============================] - ETA: 0s - loss: 0.0804 - accuracy: 0.96 - 0s 138us/sample - loss: 0.1035 - accuracy: 0.9655 - val_loss: 3.7752 - val_accuracy: 0.5517
Epoch 62/200
116/116 [==============================] - ETA: 0s - loss: 0.1041 - accuracy: 0.93 - 0s 138us/sample - loss: 0.0758 - accuracy: 0.9741 - val_loss: 3.9789 - val_accuracy: 0.6552
Epoch 63/200
116/116 [==============================] - ETA: 0s

...

116/116 [==============================] - ETA: 0s - loss: 0.3575 - accuracy: 0.87 - 0s 146us/sample - loss: 0.2435 - accuracy: 0.9310 - val_loss: 3.8456 - val_accuracy: 0.5862
Epoch 67/200
116/116 [==============================] - ETA: 0s - loss: 0.2332 - accuracy: 0.93 - 0s 249us/sample - loss: 0.2505 - accuracy: 0.8966 - val_loss: 3.2367 - val_accuracy: 0.5862
Epoch 68/200
116/116 [==============================] - ETA: 0s - loss: 0.1997 - accuracy: 0.93 - 0s 447us/sample - loss: 0.2152 - accuracy: 0.9052 - val_loss: 2.9127 - val_accuracy: 0.5517
Epoch 69/200
116/116 [==============================] - ETA: 0s - loss: 0.1579 - accuracy: 0.96 - 0s 464us/sample - loss: 0.1901 - accuracy: 0.9310 - val_loss: 3.3452 - val_accuracy: 0.5172
Epoch 70/200
116/116 [==============================] - ETA: 0s - loss: 0.1029 - accuracy: 0.96 - 0s 490us/sample - loss: 0.1712 - accuracy: 0.9397 - val_loss: 3.5598 - val_accuracy: 0.5862
Epoch 71/200
116/116 [==============================] - ETA: 0s

Epoch 152/200
116/116 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 1.00 - 0s 120us/sample - loss: 0.0041 - accuracy: 1.0000 - val_loss: 8.2311 - val_accuracy: 0.5517
Epoch 153/200
116/116 [==============================] - ETA: 0s - loss: 0.0070 - accuracy: 1.00 - 0s 112us/sample - loss: 0.0039 - accuracy: 1.0000 - val_loss: 8.3460 - val_accuracy: 0.5517
Epoch 154/200
116/116 [==============================] - ETA: 0s - loss: 0.0052 - accuracy: 1.00 - 0s 120us/sample - loss: 0.0026 - accuracy: 1.0000 - val_loss: 8.6108 - val_accuracy: 0.5517
Epoch 155/200
116/116 [==============================] - ETA: 0s - loss: 0.0014 - accuracy: 1.00 - 0s 112us/sample - loss: 0.0027 - accuracy: 1.0000 - val_loss: 8.7705 - val_accuracy: 0.5517
Epoch 156/200
116/116 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.00 - 0s 120us/sample - loss: 0.0020 - accuracy: 1.0000 - val_loss: 8.8499 - val_accuracy: 0.5517
Epoch 157/200
116/116 [======================

Epoch 194/200
116/116 [==============================] - ETA: 0s - loss: 1.7139e-04 - accuracy: 1.00 - 0s 120us/sample - loss: 3.5252e-04 - accuracy: 1.0000 - val_loss: 11.2480 - val_accuracy: 0.5517
Epoch 195/200
116/116 [==============================] - ETA: 0s - loss: 1.5679e-04 - accuracy: 1.00 - 0s 129us/sample - loss: 3.4147e-04 - accuracy: 1.0000 - val_loss: 11.2972 - val_accuracy: 0.5517
Epoch 196/200
116/116 [==============================] - ETA: 0s - loss: 1.3284e-04 - accuracy: 1.00 - 0s 129us/sample - loss: 3.3720e-04 - accuracy: 1.0000 - val_loss: 11.3506 - val_accuracy: 0.5517
Epoch 197/200
116/116 [==============================] - ETA: 0s - loss: 4.6123e-04 - accuracy: 1.00 - 0s 249us/sample - loss: 3.2689e-04 - accuracy: 1.0000 - val_loss: 11.3884 - val_accuracy: 0.5517
Epoch 198/200
116/116 [==============================] - ETA: 0s - loss: 6.9706e-04 - accuracy: 1.00 - 0s 129us/sample - loss: 3.1262e-04 - accuracy: 1.0000 - val_loss: 11.3999 - val_accuracy: 0.5517


Epoch 37/200
116/116 [==============================] - ETA: 0s - loss: 0.2830 - accuracy: 0.81 - 0s 138us/sample - loss: 0.1735 - accuracy: 0.9397 - val_loss: 1.3419 - val_accuracy: 0.6897
Epoch 38/200
116/116 [==============================] - ETA: 0s - loss: 0.3056 - accuracy: 0.81 - 0s 120us/sample - loss: 0.1923 - accuracy: 0.9138 - val_loss: 1.5820 - val_accuracy: 0.6207
Epoch 39/200
116/116 [==============================] - ETA: 0s - loss: 0.1485 - accuracy: 0.96 - 0s 146us/sample - loss: 0.1476 - accuracy: 0.9483 - val_loss: 1.7783 - val_accuracy: 0.6207
Epoch 40/200
116/116 [==============================] - ETA: 0s - loss: 0.1712 - accuracy: 0.93 - 0s 129us/sample - loss: 0.1462 - accuracy: 0.9397 - val_loss: 1.6793 - val_accuracy: 0.6552
Epoch 41/200
116/116 [==============================] - ETA: 0s - loss: 0.1570 - accuracy: 0.90 - 0s 129us/sample - loss: 0.1415 - accuracy: 0.9310 - val_loss: 1.9225 - val_accuracy: 0.5862
Epoch 42/200
116/116 [============================

Epoch 123/200
116/116 [==============================] - ETA: 0s - loss: 0.0415 - accuracy: 0.96 - 0s 137us/sample - loss: 0.0212 - accuracy: 0.9914 - val_loss: 4.8176 - val_accuracy: 0.6207
Epoch 124/200
116/116 [==============================] - ETA: 0s - loss: 0.0118 - accuracy: 1.00 - 0s 138us/sample - loss: 0.0163 - accuracy: 1.0000 - val_loss: 4.9707 - val_accuracy: 0.6207
Epoch 125/200
116/116 [==============================] - ETA: 0s - loss: 0.0228 - accuracy: 1.00 - 0s 138us/sample - loss: 0.0123 - accuracy: 1.0000 - val_loss: 5.1118 - val_accuracy: 0.5172
Epoch 126/200
116/116 [==============================] - ETA: 0s - loss: 0.0067 - accuracy: 1.00 - 0s 146us/sample - loss: 0.0256 - accuracy: 0.9914 - val_loss: 5.1868 - val_accuracy: 0.5517
Epoch 127/200
116/116 [==============================] - ETA: 0s - loss: 0.0071 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0122 - accuracy: 1.0000 - val_loss: 5.2153 - val_accuracy: 0.6207
Epoch 128/200
116/116 [======================

In [23]:
print(tuner.get_best_hyperparameters()[0].values)

print(tuner.results_summary())

{'input_units': 108, 'n_layers': 2, 'dense_0_units': 96, 'dense_1_units': 60, 'dense_2_units': 36}


None


In [26]:
best_model=tuner.get_best_models()[0]

y_pred=best_model.predict(X_test)
y_pred[y_pred<0.5]=0
y_pred[y_pred>0.5]=1
auc_nn=metrics.roc_auc_score(y_test,y_pred)
acc_nn=metrics.accuracy_score(y_test,y_pred)

print("AUC:",auc_nn," Accuracy:",acc_nn)

AUC: 0.6736842105263158  Accuracy: 0.7586206896551724


## Dataset "2 years before"

In [29]:
df2=pd.read_csv("2 years before.csv")
df2['final'][df2['final']=='Bankrupted']=0
df2['final'][df2['final']=='Non-Bankrupted']=1
df2.head()

bankruptcies=df2['final'][df2['final']==0].count()
rows=df2.shape[0]
columns=df2.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 12 columns. We have 49 bankrupted companies and 96 companies that still operate.


## Train test split and normalization

In [30]:
np.random.seed(333)

X=df2.iloc[:,0:-1].values
Y=df2.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

In [32]:
## Building the model

In [35]:
def build_model_2(hp):
    
    model = Sequential()
    
    model.add(Dense(hp.Int("input_units",min_value=11,max_value=110,step=11),input_shape=(11,),activation='relu'))
    
    for i in range(hp.Int("n_layers",1,4)):
    
        model.add(Dense(hp.Int(f"dense_{i}_units",min_value=11,max_value=110,step=11),activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(Adam(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])

    return model

# Do NOT run this

In [36]:
LOG_DIR=f"second_year\{int(time.time())}"

tuner2= RandomSearch(
    build_model_2,
    objective=kt.Objective("val_accuracy",direction='max'),
    max_trials=4,
    executions_per_trial=4,
    directory=LOG_DIR
)

earlyStopping =EarlyStopping(monitor='loss', patience=50, verbose=0, mode='min')
#mcp_save = ModelCheckpoint('mp.hdf5', save_best_only=True, monitor='loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1,min_delta=0.01, mode='min')

tuner2.search(
    x=X_train,
    y=y_train,
    validation_data=(X_test,y_test),
    epochs=200,
    callbacks=[earlyStopping]
)

Train on 116 samples, validate on 29 samples
Epoch 1/200
116/116 [==============================] - ETA: 0s - loss: 0.6901 - accuracy: 0.62 - 1s 5ms/sample - loss: 0.6705 - accuracy: 0.6638 - val_loss: 0.6397 - val_accuracy: 0.6552
Epoch 2/200
116/116 [==============================] - ETA: 0s - loss: 0.5915 - accuracy: 0.75 - 0s 232us/sample - loss: 0.6354 - accuracy: 0.6638 - val_loss: 0.6310 - val_accuracy: 0.6552
Epoch 3/200
116/116 [==============================] - ETA: 0s - loss: 0.7024 - accuracy: 0.59 - 0s 189us/sample - loss: 0.6239 - accuracy: 0.6638 - val_loss: 0.6249 - val_accuracy: 0.6552
Epoch 4/200
116/116 [==============================] - ETA: 0s - loss: 0.6271 - accuracy: 0.65 - 0s 172us/sample - loss: 0.6172 - accuracy: 0.6638 - val_loss: 0.6169 - val_accuracy: 0.6552
Epoch 5/200
116/116 [==============================] - ETA: 0s - loss: 0.5990 - accuracy: 0.68 - 0s 181us/sample - loss: 0.6108 - accuracy: 0.6638 - val_loss: 0.6083 - val_accuracy: 0.6552
Epoch 6/200


Epoch 44/200
116/116 [==============================] - ETA: 0s - loss: 0.5476 - accuracy: 0.71 - 0s 155us/sample - loss: 0.4511 - accuracy: 0.7931 - val_loss: 0.8907 - val_accuracy: 0.6897
Epoch 45/200
116/116 [==============================] - ETA: 0s - loss: 0.3768 - accuracy: 0.78 - 0s 163us/sample - loss: 0.3989 - accuracy: 0.8103 - val_loss: 0.6997 - val_accuracy: 0.7586
Epoch 46/200
116/116 [==============================] - ETA: 0s - loss: 0.4211 - accuracy: 0.81 - 0s 146us/sample - loss: 0.4145 - accuracy: 0.7845 - val_loss: 0.7539 - val_accuracy: 0.7241
Epoch 47/200
116/116 [==============================] - ETA: 0s - loss: 0.3899 - accuracy: 0.84 - 0s 138us/sample - loss: 0.3931 - accuracy: 0.8448 - val_loss: 0.7595 - val_accuracy: 0.7241
Epoch 48/200
116/116 [==============================] - ETA: 0s - loss: 0.2257 - accuracy: 1.00 - 0s 129us/sample - loss: 0.3798 - accuracy: 0.8534 - val_loss: 0.8093 - val_accuracy: 0.7241
Epoch 49/200
116/116 [============================

Epoch 130/200
116/116 [==============================] - ETA: 0s - loss: 0.1896 - accuracy: 0.84 - 0s 129us/sample - loss: 0.1521 - accuracy: 0.9052 - val_loss: 2.4455 - val_accuracy: 0.7241
Epoch 131/200
116/116 [==============================] - ETA: 0s - loss: 0.1781 - accuracy: 0.90 - 0s 155us/sample - loss: 0.1302 - accuracy: 0.9397 - val_loss: 2.3622 - val_accuracy: 0.7586
Epoch 132/200
116/116 [==============================] - ETA: 0s - loss: 0.1172 - accuracy: 0.96 - 0s 138us/sample - loss: 0.1284 - accuracy: 0.9655 - val_loss: 2.4250 - val_accuracy: 0.6897
Epoch 133/200
116/116 [==============================] - ETA: 0s - loss: 0.1581 - accuracy: 0.90 - 0s 129us/sample - loss: 0.1224 - accuracy: 0.9310 - val_loss: 2.5609 - val_accuracy: 0.7586
Epoch 134/200
116/116 [==============================] - ETA: 0s - loss: 0.0411 - accuracy: 1.00 - 0s 146us/sample - loss: 0.1589 - accuracy: 0.9310 - val_loss: 2.6169 - val_accuracy: 0.6897
Epoch 135/200
116/116 [======================

Epoch 173/200
116/116 [==============================] - ETA: 0s - loss: 0.0807 - accuracy: 0.93 - 0s 146us/sample - loss: 0.1057 - accuracy: 0.9397 - val_loss: 3.2466 - val_accuracy: 0.6897
Epoch 174/200
116/116 [==============================] - ETA: 0s - loss: 0.1245 - accuracy: 0.93 - 0s 146us/sample - loss: 0.0876 - accuracy: 0.9569 - val_loss: 3.1438 - val_accuracy: 0.6897
Epoch 175/200
116/116 [==============================] - ETA: 0s - loss: 0.0555 - accuracy: 1.00 - 0s 129us/sample - loss: 0.0964 - accuracy: 0.9655 - val_loss: 3.1325 - val_accuracy: 0.6207
Epoch 176/200
116/116 [==============================] - ETA: 0s - loss: 0.0642 - accuracy: 1.00 - 0s 129us/sample - loss: 0.1038 - accuracy: 0.9569 - val_loss: 3.2426 - val_accuracy: 0.6897
Epoch 177/200
116/116 [==============================] - ETA: 0s - loss: 0.0832 - accuracy: 0.96 - 0s 120us/sample - loss: 0.0869 - accuracy: 0.9655 - val_loss: 3.3740 - val_accuracy: 0.6897
Epoch 178/200
116/116 [======================

Epoch 16/200
116/116 [==============================] - ETA: 0s - loss: 0.5548 - accuracy: 0.81 - 0s 155us/sample - loss: 0.5693 - accuracy: 0.6983 - val_loss: 0.5850 - val_accuracy: 0.6552
Epoch 17/200
116/116 [==============================] - ETA: 0s - loss: 0.5003 - accuracy: 0.81 - 0s 138us/sample - loss: 0.5663 - accuracy: 0.7328 - val_loss: 0.5779 - val_accuracy: 0.7586
Epoch 18/200
116/116 [==============================] - ETA: 0s - loss: 0.5363 - accuracy: 0.75 - 0s 129us/sample - loss: 0.5513 - accuracy: 0.7241 - val_loss: 0.5654 - val_accuracy: 0.6552
Epoch 19/200
116/116 [==============================] - ETA: 0s - loss: 0.5848 - accuracy: 0.71 - 0s 129us/sample - loss: 0.5425 - accuracy: 0.7241 - val_loss: 0.5697 - val_accuracy: 0.6552
Epoch 20/200
116/116 [==============================] - ETA: 0s - loss: 0.4907 - accuracy: 0.81 - 0s 138us/sample - loss: 0.5345 - accuracy: 0.7672 - val_loss: 0.5635 - val_accuracy: 0.6552
Epoch 21/200
116/116 [============================

...

116/116 [==============================] - ETA: 0s - loss: 0.5183 - accuracy: 0.68 - 0s 155us/sample - loss: 0.3565 - accuracy: 0.8362 - val_loss: 2.9873 - val_accuracy: 0.7241
Epoch 118/200
116/116 [==============================] - ETA: 0s - loss: 0.2481 - accuracy: 0.93 - 0s 155us/sample - loss: 0.3087 - accuracy: 0.8621 - val_loss: 2.5388 - val_accuracy: 0.7586
Epoch 119/200
116/116 [==============================] - ETA: 0s - loss: 0.3867 - accuracy: 0.81 - 0s 146us/sample - loss: 0.3062 - accuracy: 0.8707 - val_loss: 3.3888 - val_accuracy: 0.7586
Epoch 120/200
116/116 [==============================] - ETA: 0s - loss: 0.2674 - accuracy: 0.87 - 0s 164us/sample - loss: 0.3114 - accuracy: 0.8707 - val_loss: 3.3798 - val_accuracy: 0.7241
Epoch 121/200
116/116 [==============================] - ETA: 0s - loss: 0.4081 - accuracy: 0.78 - 0s 138us/sample - loss: 0.3182 - accuracy: 0.8448 - val_loss: 3.8860 - val_accuracy: 0.6897
Epoch 122/200
116/116 [==============================] - ET

Epoch 160/200
116/116 [==============================] - ETA: 0s - loss: 0.2319 - accuracy: 0.87 - 0s 155us/sample - loss: 0.1795 - accuracy: 0.9224 - val_loss: 3.8158 - val_accuracy: 0.7241
Epoch 161/200
116/116 [==============================] - ETA: 0s - loss: 0.2441 - accuracy: 0.84 - 0s 146us/sample - loss: 0.1696 - accuracy: 0.9224 - val_loss: 4.3086 - val_accuracy: 0.6552
Epoch 162/200
116/116 [==============================] - ETA: 0s - loss: 0.1394 - accuracy: 0.96 - 0s 146us/sample - loss: 0.1539 - accuracy: 0.9397 - val_loss: 4.7571 - val_accuracy: 0.7241
Epoch 163/200
116/116 [==============================] - ETA: 0s - loss: 0.1191 - accuracy: 0.93 - 0s 155us/sample - loss: 0.1357 - accuracy: 0.9310 - val_loss: 5.2279 - val_accuracy: 0.6897
Epoch 164/200
116/116 [==============================] - ETA: 0s - loss: 0.1389 - accuracy: 0.93 - 0s 129us/sample - loss: 0.1364 - accuracy: 0.9397 - val_loss: 5.9860 - val_accuracy: 0.6552
Epoch 165/200
116/116 [======================

Epoch 3/200
116/116 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.56 - 0s 172us/sample - loss: 0.6384 - accuracy: 0.6638 - val_loss: 0.6210 - val_accuracy: 0.6207
Epoch 4/200
116/116 [==============================] - ETA: 0s - loss: 0.6201 - accuracy: 0.62 - 0s 189us/sample - loss: 0.6080 - accuracy: 0.6810 - val_loss: 0.6073 - val_accuracy: 0.6552
Epoch 5/200
116/116 [==============================] - ETA: 0s - loss: 0.7194 - accuracy: 0.62 - 0s 181us/sample - loss: 0.6097 - accuracy: 0.6724 - val_loss: 0.5948 - val_accuracy: 0.7241
Epoch 6/200
116/116 [==============================] - ETA: 0s - loss: 0.6298 - accuracy: 0.65 - 0s 172us/sample - loss: 0.5919 - accuracy: 0.7155 - val_loss: 0.5926 - val_accuracy: 0.6552
Epoch 7/200
116/116 [==============================] - ETA: 0s - loss: 0.6873 - accuracy: 0.62 - 0s 181us/sample - loss: 0.5767 - accuracy: 0.7069 - val_loss: 0.5864 - val_accuracy: 0.6897
Epoch 8/200
116/116 [==============================] - 

116/116 [==============================] - ETA: 0s - loss: 0.3015 - accuracy: 0.78 - 0s 129us/sample - loss: 0.3333 - accuracy: 0.8362 - val_loss: 1.4095 - val_accuracy: 0.7586
Epoch 90/200
116/116 [==============================] - ETA: 0s - loss: 0.3193 - accuracy: 0.84 - 0s 138us/sample - loss: 0.3215 - accuracy: 0.8362 - val_loss: 1.5241 - val_accuracy: 0.7931
Epoch 91/200
116/116 [==============================] - ETA: 0s - loss: 0.2971 - accuracy: 0.84 - 0s 129us/sample - loss: 0.3492 - accuracy: 0.7845 - val_loss: 1.4909 - val_accuracy: 0.6207
Epoch 92/200
116/116 [==============================] - ETA: 0s - loss: 0.3594 - accuracy: 0.84 - 0s 129us/sample - loss: 0.3374 - accuracy: 0.8017 - val_loss: 0.9873 - val_accuracy: 0.7241
Epoch 93/200
116/116 [==============================] - ETA: 0s - loss: 0.3496 - accuracy: 0.78 - 0s 129us/sample - loss: 0.3908 - accuracy: 0.8276 - val_loss: 0.9655 - val_accuracy: 0.7931
Epoch 94/200
116/116 [==============================] - ETA: 0s

Epoch 175/200
116/116 [==============================] - ETA: 0s - loss: 0.2132 - accuracy: 0.90 - 0s 129us/sample - loss: 0.2669 - accuracy: 0.8793 - val_loss: 4.3613 - val_accuracy: 0.7586
Epoch 176/200
116/116 [==============================] - ETA: 0s - loss: 0.1827 - accuracy: 0.93 - 0s 138us/sample - loss: 0.1994 - accuracy: 0.9052 - val_loss: 4.3134 - val_accuracy: 0.7586
Epoch 177/200
116/116 [==============================] - ETA: 0s - loss: 0.2113 - accuracy: 0.93 - 0s 155us/sample - loss: 0.1657 - accuracy: 0.9224 - val_loss: 4.6272 - val_accuracy: 0.7241
Epoch 178/200
116/116 [==============================] - ETA: 0s - loss: 0.1480 - accuracy: 0.93 - 0s 146us/sample - loss: 0.1542 - accuracy: 0.9052 - val_loss: 4.7410 - val_accuracy: 0.7931
Epoch 179/200
116/116 [==============================] - ETA: 0s - loss: 0.1383 - accuracy: 0.90 - 0s 146us/sample - loss: 0.1672 - accuracy: 0.8966 - val_loss: 4.7996 - val_accuracy: 0.7586
Epoch 180/200
116/116 [======================

In [37]:
print(tuner2.get_best_hyperparameters()[0].values)

print(tuner2.results_summary())

{'input_units': 66, 'n_layers': 4, 'dense_0_units': 44, 'dense_1_units': 77, 'dense_2_units': 11, 'dense_3_units': 11}


None


In [39]:
best_model2=tuner2.get_best_models()[0]

y_pred=best_model2.predict(X_test)
y_pred[y_pred<0.5]=0
y_pred[y_pred>0.5]=1
auc_nn2=metrics.roc_auc_score(y_test,y_pred)
acc_nn2=metrics.accuracy_score(y_test,y_pred)

print("AUC:",auc_nn2," Accuracy:",acc_nn2)

AUC: 0.7973684210526315  Accuracy: 0.8275862068965517


## Dataset "last year"

In [40]:
df_last=pd.read_csv("last year.csv")
df_last['final'][df_last['final']=='Bankrupted']=0
df_last['final'][df_last['final']=='Non-Bankrupted']=1
df_last.head()

bankruptcies=df_last['final'][df_last['final']==0].count()
rows=df_last.shape[0]
columns=df_last.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 38 columns. We have 49 bankrupted companies and 96 companies that still operate.


## Train test split and normalization

In [41]:
np.random.seed(333)

X=df_last.iloc[:,0:-1].values
Y=df_last.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

## Building the model

In [42]:
def build_model_last(hp):
    
    model = Sequential()
    
    model.add(Dense(hp.Int("input_units",min_value=37,max_value=167,step=13),input_shape=(37,),activation='relu'))
    
    for i in range(hp.Int("n_layers",1,4)):
    
        model.add(Dense(hp.Int(f"dense_{i}_units",min_value=24,max_value=120,step=12),activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(Adam(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])

    return model

# Do NOT run this

In [43]:
LOG_DIR=f"last_year\{int(time.time())}"

tuner_last= RandomSearch(
    build_model_last,
    objective=kt.Objective("val_accuracy",direction='max'),
    max_trials=4,
    executions_per_trial=4,
    directory=LOG_DIR
)

earlyStopping =EarlyStopping(monitor='loss', patience=50, verbose=0, mode='min')
#mcp_save = ModelCheckpoint('mp.hdf5', save_best_only=True, monitor='loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1,min_delta=0.01, mode='min')

tuner_last.search(
    x=X_train,
    y=y_train,
    validation_data=(X_test,y_test),
    epochs=200,
    callbacks=[earlyStopping]
)

W0316 10:20:33.108121 13836 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0316 10:20:33.110089 13836 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0316 10:20:33.112084 13836 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0316 10:20:33.114077 13836 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0316 10:20:33.115076 13836 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0316 10:20:33.118065 13836 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


Train on 116 samples, validate on 29 samples
Epoch 1/200
116/116 [==============================] - ETA: 0s - loss: 0.6870 - accuracy: 0.62 - 1s 6ms/sample - loss: 0.6795 - accuracy: 0.6552 - val_loss: 0.6466 - val_accuracy: 0.6552
Epoch 2/200
116/116 [==============================] - ETA: 0s - loss: 0.6772 - accuracy: 0.62 - 0s 301us/sample - loss: 0.6660 - accuracy: 0.6638 - val_loss: 0.6542 - val_accuracy: 0.6552
Epoch 3/200
116/116 [==============================] - ETA: 0s - loss: 0.6150 - accuracy: 0.75 - 0s 180us/sample - loss: 0.6324 - accuracy: 0.6638 - val_loss: 0.6323 - val_accuracy: 0.6552
Epoch 4/200
116/116 [==============================] - ETA: 0s - loss: 0.6068 - accuracy: 0.68 - 0s 189us/sample - loss: 0.6202 - accuracy: 0.6638 - val_loss: 0.6379 - val_accuracy: 0.6552
Epoch 5/200
116/116 [==============================] - ETA: 0s - loss: 0.5772 - accuracy: 0.71 - 0s 189us/sample - loss: 0.6131 - accuracy: 0.6638 - val_loss: 0.6222 - val_accuracy: 0.6552
Epoch 6/200


Epoch 44/200
116/116 [==============================] - ETA: 0s - loss: 0.1653 - accuracy: 0.96 - 0s 146us/sample - loss: 0.1417 - accuracy: 0.9483 - val_loss: 2.0581 - val_accuracy: 0.6207
Epoch 45/200
116/116 [==============================] - ETA: 0s - loss: 0.0729 - accuracy: 0.96 - 0s 146us/sample - loss: 0.1273 - accuracy: 0.9569 - val_loss: 2.4837 - val_accuracy: 0.5172
Epoch 46/200
116/116 [==============================] - ETA: 0s - loss: 0.0618 - accuracy: 1.00 - 0s 146us/sample - loss: 0.1057 - accuracy: 0.9828 - val_loss: 2.7799 - val_accuracy: 0.5862
Epoch 47/200
116/116 [==============================] - ETA: 0s - loss: 0.0494 - accuracy: 1.00 - 0s 138us/sample - loss: 0.0947 - accuracy: 0.9655 - val_loss: 3.0006 - val_accuracy: 0.5862
Epoch 48/200
116/116 [==============================] - ETA: 0s - loss: 0.0504 - accuracy: 1.00 - 0s 189us/sample - loss: 0.0876 - accuracy: 0.9741 - val_loss: 2.9424 - val_accuracy: 0.6207
Epoch 49/200
116/116 [============================

Epoch 129/200
116/116 [==============================] - ETA: 0s - loss: 1.4809e-04 - accuracy: 1.00 - 0s 146us/sample - loss: 1.4796e-04 - accuracy: 1.0000 - val_loss: 7.6093 - val_accuracy: 0.5862
Epoch 130/200
116/116 [==============================] - ETA: 0s - loss: 2.4624e-04 - accuracy: 1.00 - 0s 146us/sample - loss: 1.4402e-04 - accuracy: 1.0000 - val_loss: 7.6187 - val_accuracy: 0.5517
Epoch 131/200
116/116 [==============================] - ETA: 0s - loss: 2.1375e-04 - accuracy: 1.00 - 0s 155us/sample - loss: 1.4168e-04 - accuracy: 1.0000 - val_loss: 7.6260 - val_accuracy: 0.5517
Epoch 132/200
116/116 [==============================] - ETA: 0s - loss: 8.6166e-05 - accuracy: 1.00 - 0s 155us/sample - loss: 1.3637e-04 - accuracy: 1.0000 - val_loss: 7.6313 - val_accuracy: 0.5517
Epoch 133/200
116/116 [==============================] - ETA: 0s - loss: 1.4136e-04 - accuracy: 1.00 - 0s 155us/sample - loss: 1.3286e-04 - accuracy: 1.0000 - val_loss: 7.6339 - val_accuracy: 0.5517
Epoch

Epoch 12/200
116/116 [==============================] - ETA: 0s - loss: 0.3626 - accuracy: 0.84 - 0s 172us/sample - loss: 0.4594 - accuracy: 0.7759 - val_loss: 0.7120 - val_accuracy: 0.6207
Epoch 13/200
116/116 [==============================] - ETA: 0s - loss: 0.4544 - accuracy: 0.81 - 0s 181us/sample - loss: 0.4037 - accuracy: 0.8534 - val_loss: 0.7430 - val_accuracy: 0.6207
Epoch 14/200
116/116 [==============================] - ETA: 0s - loss: 0.4076 - accuracy: 0.87 - 0s 138us/sample - loss: 0.3952 - accuracy: 0.8448 - val_loss: 0.8663 - val_accuracy: 0.6552
Epoch 15/200
116/116 [==============================] - ETA: 0s - loss: 0.3152 - accuracy: 0.90 - 0s 138us/sample - loss: 0.3921 - accuracy: 0.8621 - val_loss: 0.8724 - val_accuracy: 0.5517
Epoch 16/200
116/116 [==============================] - ETA: 0s - loss: 0.3448 - accuracy: 0.84 - 0s 138us/sample - loss: 0.4035 - accuracy: 0.8103 - val_loss: 0.8555 - val_accuracy: 0.6897
Epoch 17/200
116/116 [============================

...

Epoch 19/200
116/116 [==============================] - ETA: 0s - loss: 0.2868 - accuracy: 0.93 - 0s 146us/sample - loss: 0.2372 - accuracy: 0.8966 - val_loss: 1.2185 - val_accuracy: 0.6207
Epoch 20/200
116/116 [==============================] - ETA: 0s - loss: 0.2143 - accuracy: 0.96 - 0s 138us/sample - loss: 0.2158 - accuracy: 0.9224 - val_loss: 1.3621 - val_accuracy: 0.5862
Epoch 21/200
116/116 [==============================] - ETA: 0s - loss: 0.1731 - accuracy: 0.96 - 0s 301us/sample - loss: 0.2075 - accuracy: 0.9397 - val_loss: 1.1128 - val_accuracy: 0.6897
Epoch 22/200
116/116 [==============================] - ETA: 0s - loss: 0.2872 - accuracy: 0.84 - 0s 146us/sample - loss: 0.2378 - accuracy: 0.8966 - val_loss: 1.6000 - val_accuracy: 0.5517
Epoch 23/200
116/116 [==============================] - ETA: 0s - loss: 0.2850 - accuracy: 0.78 - 0s 146us/sample - loss: 0.2069 - accuracy: 0.8879 - val_loss: 1.7033 - val_accuracy: 0.6552
Epoch 24/200
116/116 [============================

116/116 [==============================] - ETA: 0s - loss: 1.6090e-04 - accuracy: 1.00 - 0s 155us/sample - loss: 1.8691e-04 - accuracy: 1.0000 - val_loss: 5.7221 - val_accuracy: 0.6207
Epoch 104/200
116/116 [==============================] - ETA: 0s - loss: 2.0923e-04 - accuracy: 1.00 - 0s 146us/sample - loss: 1.8181e-04 - accuracy: 1.0000 - val_loss: 5.7286 - val_accuracy: 0.6207
Epoch 105/200
116/116 [==============================] - ETA: 0s - loss: 1.9832e-04 - accuracy: 1.00 - 0s 146us/sample - loss: 1.7792e-04 - accuracy: 1.0000 - val_loss: 5.7340 - val_accuracy: 0.6207
Epoch 106/200
116/116 [==============================] - ETA: 0s - loss: 9.1234e-05 - accuracy: 1.00 - 0s 129us/sample - loss: 1.7299e-04 - accuracy: 1.0000 - val_loss: 5.7460 - val_accuracy: 0.6207
Epoch 107/200
116/116 [==============================] - ETA: 0s - loss: 1.4770e-04 - accuracy: 1.00 - 0s 146us/sample - loss: 1.6868e-04 - accuracy: 1.0000 - val_loss: 5.7666 - val_accuracy: 0.6207
Epoch 108/200
116/1

116/116 [==============================] - ETA: 0s - loss: 5.5937e-05 - accuracy: 1.00 - 0s 464us/sample - loss: 4.5377e-05 - accuracy: 1.0000 - val_loss: 6.5263 - val_accuracy: 0.6207
Epoch 186/200
116/116 [==============================] - ETA: 0s - loss: 3.8125e-05 - accuracy: 1.00 - 0s 473us/sample - loss: 4.4795e-05 - accuracy: 1.0000 - val_loss: 6.5338 - val_accuracy: 0.6207
Epoch 187/200
116/116 [==============================] - ETA: 0s - loss: 3.4140e-05 - accuracy: 1.00 - 0s 524us/sample - loss: 4.4333e-05 - accuracy: 1.0000 - val_loss: 6.5409 - val_accuracy: 0.6207
Epoch 188/200
116/116 [==============================] - ETA: 0s - loss: 5.7805e-05 - accuracy: 1.00 - 0s 155us/sample - loss: 4.3699e-05 - accuracy: 1.0000 - val_loss: 6.5474 - val_accuracy: 0.6207
Epoch 189/200
116/116 [==============================] - ETA: 0s - loss: 3.1866e-05 - accuracy: 1.00 - 0s 146us/sample - loss: 4.2977e-05 - accuracy: 1.0000 - val_loss: 6.5517 - val_accuracy: 0.6207
Epoch 190/200
116/1

Epoch 70/200
116/116 [==============================] - ETA: 0s - loss: 1.9364e-04 - accuracy: 1.00 - 0s 155us/sample - loss: 2.9922e-04 - accuracy: 1.0000 - val_loss: 4.5134 - val_accuracy: 0.5517
Epoch 71/200
116/116 [==============================] - ETA: 0s - loss: 3.3470e-04 - accuracy: 1.00 - 0s 138us/sample - loss: 2.7872e-04 - accuracy: 1.0000 - val_loss: 4.5582 - val_accuracy: 0.5517
Epoch 72/200
116/116 [==============================] - ETA: 0s - loss: 3.1553e-04 - accuracy: 1.00 - 0s 138us/sample - loss: 2.6051e-04 - accuracy: 1.0000 - val_loss: 4.5983 - val_accuracy: 0.5517
Epoch 73/200
116/116 [==============================] - ETA: 0s - loss: 3.3373e-04 - accuracy: 1.00 - 0s 138us/sample - loss: 2.4517e-04 - accuracy: 1.0000 - val_loss: 4.6417 - val_accuracy: 0.5517
Epoch 74/200
116/116 [==============================] - ETA: 0s - loss: 1.9192e-04 - accuracy: 1.00 - 0s 138us/sample - loss: 2.2852e-04 - accuracy: 1.0000 - val_loss: 4.6817 - val_accuracy: 0.5517
Epoch 75/2

116/116 [==============================] - ETA: 0s - loss: 2.7520e-05 - accuracy: 1.00 - 0s 499us/sample - loss: 2.5350e-05 - accuracy: 1.0000 - val_loss: 6.0472 - val_accuracy: 0.5517
Epoch 153/200
116/116 [==============================] - ETA: 0s - loss: 4.0475e-05 - accuracy: 1.00 - 0s 473us/sample - loss: 2.4911e-05 - accuracy: 1.0000 - val_loss: 6.0569 - val_accuracy: 0.5517
Epoch 154/200
116/116 [==============================] - ETA: 0s - loss: 3.4110e-05 - accuracy: 1.00 - 0s 499us/sample - loss: 2.4507e-05 - accuracy: 1.0000 - val_loss: 6.0663 - val_accuracy: 0.5517
Epoch 155/200
116/116 [==============================] - ETA: 0s - loss: 1.9651e-05 - accuracy: 1.00 - 0s 464us/sample - loss: 2.4171e-05 - accuracy: 1.0000 - val_loss: 6.0752 - val_accuracy: 0.5517
Epoch 156/200
116/116 [==============================] - ETA: 0s - loss: 3.2869e-05 - accuracy: 1.00 - 0s 490us/sample - loss: 2.3902e-05 - accuracy: 1.0000 - val_loss: 6.0855 - val_accuracy: 0.5517
Epoch 157/200
116/1

In [44]:
print(tuner_last.get_best_hyperparameters()[0].values)

print(tuner_last.results_summary())

{'input_units': 154, 'n_layers': 2, 'dense_0_units': 48, 'dense_1_units': 120, 'dense_2_units': 72}


None


In [45]:
best_model_last=tuner_last.get_best_models()[0]

y_pred=best_model_last.predict(X_test)
y_pred[y_pred<0.5]=0
y_pred[y_pred>0.5]=1
auc_nn_last=metrics.roc_auc_score(y_test,y_pred)
acc_nn_last=metrics.accuracy_score(y_test,y_pred)

print("AUC:",auc_nn_last," Accuracy:",acc_nn_last)

AUC: 0.7236842105263157  Accuracy: 0.7931034482758621


## All results

In [46]:
data={'Deep Neural Network_AUC':[auc_nn,auc_nn2,auc_nn_last],
     'Deep Neural Network_ACC':[acc_nn,acc_nn2,acc_nn_last]}

results=pd.DataFrame(data,index=['1 years before','2 years before','last year'])

results

,Deep Neural Network_AUC,Deep Neural Network_ACC
1 years before,0.673684,0.758621
2 years before,0.797368,0.827586
last year,0.723684,0.793103
